In [1]:
import pandas as pd

In [2]:
schedule = pd.read_csv('../INPUT/data_fam_schedule.csv')

In [3]:
schedule

,flight,deptime,depoff,arrtime,arroff,origin,destination,Depart_time,Arrive_time,Flight_time
0,AA0001,755,-400,1038,-700,EDB,TFB,1155,1738,583
1,AA0002,1405,-700,2159,-400,TFB,EDB,2105,2559,454
2,AA0003,1740,-400,2025,-700,EDB,TFB,2140,2725,585
3,AA0004,840,-700,1640,-400,TFB,EDB,1540,2040,500
4,AA0005,910,-400,1202,-700,EDB,MBY,1310,1902,592
...,...,...,...,...,...,...,...,...,...,...
810,BA2645,1705,-700,1850,-700,QEY,TNG,2405,2550,145
811,BA2670,1335,-700,1435,-700,TFB,ZMX,2035,2135,100
812,BA2700,2016,-700,2230,-700,NGS,MBY,2716,2930,214
813,AA2701,645,-800,1243,-700,BOD,MBT,1445,1943,498


In [4]:
# 获得经停航班号及索引
via_stop_flight = []  # 经停航班号
via_stop_index = []  # 经停航班索引

flight_list = schedule['flight'].tolist()
for i in range(len(schedule)):
    flight = flight_list[i]
    if flight in via_stop_flight:
        via_stop_index.append(i)
        continue
    if flight in flight_list[(i+1):]:
        via_stop_index.append(i)
        via_stop_flight.append(flight)

In [5]:
# 分离经停及非经停航班df

non_via_stop = schedule.copy()
via_stop = schedule.copy()

for i in via_stop_index:  # 非经停航班
    non_via_stop.drop(i, axis=0, inplace=True)

for i in  range(len(schedule)):  # 经停航班
    if i not in via_stop_index:
        via_stop.drop(i, axis=0, inplace=True)

via_stop = via_stop.reset_index(drop=True)
non_via_stop = non_via_stop.reset_index(drop=True)


In [6]:
# 对同一经停航班按起飞时间排序
# via_stop1 为最后输出
via_stop1 = pd.DataFrame()
via_stop_flight = via_stop.flight.tolist()
via_stop_flight = set(via_stop_flight)

for flight in via_stop_flight:
    flight_df = via_stop[via_stop['flight'] == flight]
    flight_df = flight_df.sort_values(by='Depart_time', ascending=True)
    via_stop1 = pd.concat([via_stop1, flight_df])

via_stop1 = via_stop1.reset_index(drop=True)

In [7]:
# 获取经停航班 i 的终到机场的函数
def getFinal(j, final=None):
    f_flight = via_stop1.loc[j, 'flight']
    if j != 128 and (f_flight == via_stop1.loc[j+1, 'flight']):
        final = getFinal(j+1, final)
    if final != None:
        return final
    final = via_stop1.loc[j, 'destination']
    return  final

In [8]:
# 对via_stop1即排序后的经停航班表整理，结果为每个经停航班合为一行，仅保留起点及终到机场
checked = []
for i in range(len(via_stop1)):
    flight = via_stop1.loc[i, 'flight']
    if flight in checked:
        via_stop1.drop(i, axis=0, inplace=True)
        continue
    destination = getFinal(i)
    via_stop1.loc[i, 'destination'] = destination
    checked.append(flight)
    
via_stop1 = via_stop1.reset_index(drop=True)

In [9]:
# 将经停与非经停表合并
flights = pd.concat([non_via_stop, via_stop1])

In [10]:
# 将起点及目的地取出
origin = flights['origin'].tolist()
destination = flights['destination'].tolist()

In [11]:
# 统计每个机场的起飞降落次数，格式为：'机场':'次数'
origin_count = dict.fromkeys(origin, 0)
destination_count = dict.fromkeys(destination, 0)

for i in origin:
    origin_count[i] += 1
for i in destination:
    destination_count[i] += 1

In [12]:
# 将同一机场的起飞降落数据合并， 格式为：'机场':('起飞次数', '降落次数')
dep_arr = dict.fromkeys(origin_count, 0)
for i in origin_count.keys():
    if i not in destination_count:
        dep_arr[i] = (origin_count[i], 0)
        continue
    dep_arr[i] = (origin_count[i], destination_count[i])
for i in destination_count.keys():
    if i in dep_arr:
        continue
    dep_arr[i] = (0, destination_count[i])

In [13]:
dep_arr  # (起飞，降落)

{'EDB': (3, 3),
 'TFB': (243, 243),
 'MBY': (35, 35),
 'FXS': (2, 2),
 'CPT': (3, 3),
 'NJB': (1, 1),
 'NDP': (1, 1),
 'PSE': (5, 5),
 'QEY': (91, 91),
 'NTQ': (2, 2),
 'BER': (2, 2),
 'BOD': (45, 45),
 'CFU': (3, 3),
 'TDD': (1, 1),
 'KOV': (4, 4),
 'GBJ': (10, 10),
 'NAU': (1, 1),
 'NFY': (2, 2),
 'HEM': (3, 3),
 'TKE': (4, 4),
 'QWS': (2, 2),
 'TGP': (8, 8),
 'TKD': (14, 14),
 'PBL': (13, 13),
 'TNG': (8, 8),
 'TBO': (12, 12),
 'TOB': (11, 11),
 'CVS': (6, 6),
 'POU': (6, 6),
 'QTQ': (4, 4),
 'MBT': (15, 15),
 'CMJ': (9, 9),
 'QIY': (7, 7),
 'UVT': (1, 1),
 'EGX': (3, 3),
 'BVT': (1, 1),
 'EFO': (3, 3),
 'HFH': (23, 23),
 'ZWS': (13, 13),
 'BUM': (1, 1),
 'TUM': (1, 1),
 'NDJ': (1, 1),
 'LPB': (3, 3),
 'PHH': (7, 7),
 'IOM': (8, 8),
 'MJI': (3, 3),
 'EVU': (3, 3),
 'MHC': (4, 4),
 'TCB': (1, 1),
 'GBU': (3, 3),
 'ZLN': (3, 3),
 'SEN': (6, 6),
 'FVH': (6, 6),
 'FBU': (3, 3),
 'BMX': (2, 2),
 'NGS': (7, 7),
 'QTD': (6, 6),
 'ZZD': (3, 3),
 'CPJ': (16, 16),
 'NNI': (1, 1),
 'QVX': (2, 